<a href="https://colab.research.google.com/github/lescardone/deep-learning-project/blob/main/CNN_transfer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, shutil
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [55]:
# Directories for our training,
# validation and test splits
base_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small'

train_dir = os.path.join(base_dir, 'train')

validation_dir = os.path.join(base_dir, 'validation')

test_dir = os.path.join(base_dir, 'test')

small_test = os.path.join(base_dir, 'small_test')

In [5]:
# Directory with our training smiling pictures
train_happy_dir = os.path.join(train_dir, 'happy')
# os.mkdir(train_happy_dir)

# Directory with our training sad pictures
train_sad_dir = os.path.join(train_dir, 'sad')
# os.mkdir(train_sad_dir)

# Directory with our validation smiling pictures
validation_happy_dir = os.path.join(validation_dir, 'happy')
# os.mkdir(validation_happy_dir)

# Directory with our validation sad pictures
validation_sad_dir = os.path.join(validation_dir, 'sad')
# os.mkdir(validation_sad_dir)

# Directory with our validation smiling pictures
test_happy_dir = os.path.join(test_dir, 'happy')
# os.mkdir(test_happy_dir)

# Directory with our validation sad pictures
test_sad_dir = os.path.join(test_dir, 'sad')
# os.mkdir(test_sad_dir)

In [11]:
### from image augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# This is module with image preprocessing utilities
from keras.preprocessing import image

fnames = [os.path.join(train_happy_dir, fname) for fname in os.listdir(train_happy_dir)]

# We pick one image to "augment"
img_path = fnames[2]

# Read the image and resize it
img = image.load_img(img_path, target_size=(224, 224))

# Convert it to a Numpy array with shape (224, 224, 3)
x = image.img_to_array(img)

# Reshape it to (1, 224, 224, 3)
x = x.reshape((1,) + x.shape)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 224,224
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 1000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [85]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle=False)

Found 652 images belonging to 2 classes.


In [7]:
from tensorflow.keras.applications import MobileNet
conv_mobile = MobileNet(weights='imagenet',
                        include_top=False,
                        input_shape=(224,224,3))

17227776/17225924 [==============================] - 1s 0us/step


In [8]:
from keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam

In [12]:
model = Sequential()
model.add(conv_mobile)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
conv_mobile.trainable = False

model.compile(optimizer=RMSprop(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=50,
                    batch_size=20,
                    validation_data=(validation_generator))

Epoch 1/50
32/32 [==============================] - 522s 16s/step - loss: 1.1778 - acc: 0.5699 - val_loss: 0.6358 - val_acc: 0.6900
Epoch 2/50
32/32 [==============================] - 75s 2s/step - loss: 0.8660 - acc: 0.6249 - val_loss: 0.6142 - val_acc: 0.7017
Epoch 3/50
32/32 [==============================] - 75s 2s/step - loss: 0.6582 - acc: 0.6966 - val_loss: 0.5715 - val_acc: 0.7350
Epoch 4/50
32/32 [==============================] - 74s 2s/step - loss: 0.5984 - acc: 0.7045 - val_loss: 0.5456 - val_acc: 0.7517
Epoch 5/50
32/32 [==============================] - 92s 3s/step - loss: 0.6241 - acc: 0.7068 - val_loss: 0.5583 - val_acc: 0.7467
Epoch 6/50
32/32 [==============================] - 76s 2s/step - loss: 0.5576 - acc: 0.7265 - val_loss: 0.5716 - val_acc: 0.7450
Epoch 7/50
32/32 [==============================] - 92s 3s/step - loss: 0.5089 - acc: 0.7765 - val_loss: 0.5953 - val_acc: 0.7283
Epoch 8/50
32/32 [==============================] - 74s 2s/step - loss: 0.5036 - acc: 0.

In [15]:
from keras.callbacks import EarlyStopping

In [16]:
callback = EarlyStopping(monitor='loss', patience=3)

In [19]:
model2 = Sequential()
model2.add(conv_mobile)
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
conv_mobile.trainable = False

model2.compile(optimizer=Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model2.fit(train_generator,
                    epochs=100,
                    batch_size=20,
                    callbacks=[callback],
                    validation_data=(validation_generator))

Epoch 1/100
32/32 [==============================] - 77s 2s/step - loss: 1.1844 - acc: 0.5270 - val_loss: 0.6932 - val_acc: 0.6533
Epoch 2/100
32/32 [==============================] - 74s 2s/step - loss: 0.7770 - acc: 0.6267 - val_loss: 0.6034 - val_acc: 0.7050
Epoch 3/100
32/32 [==============================] - 91s 3s/step - loss: 0.6053 - acc: 0.6830 - val_loss: 0.6758 - val_acc: 0.6633
Epoch 4/100
32/32 [==============================] - 74s 2s/step - loss: 0.6107 - acc: 0.6979 - val_loss: 0.5569 - val_acc: 0.6967
Epoch 5/100
32/32 [==============================] - 73s 2s/step - loss: 0.5633 - acc: 0.7229 - val_loss: 0.5602 - val_acc: 0.7233
Epoch 6/100
32/32 [==============================] - 91s 3s/step - loss: 0.5312 - acc: 0.7492 - val_loss: 0.5540 - val_acc: 0.7200
Epoch 7/100
32/32 [==============================] - 73s 2s/step - loss: 0.4842 - acc: 0.7895 - val_loss: 0.5520 - val_acc: 0.7350
Epoch 8/100
32/32 [==============================] - 73s 2s/step - loss: 0.4930 - a

In [44]:
keras_model_path = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/june10_6pm_augmentANDtransfer_mobile_adam_1'

In [46]:
from keras.models import load_model
model2.save('/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/june10_6pm_augmentANDtransfer_mobile_adam_1')

INFO:tensorflow:Assets written to: /content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/june10_6pm_augmentANDtransfer_mobile_adam_1/assets


In [48]:
restored_keras_model = load_model(keras_model_path)

In [51]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = restored_keras_model.evaluate(test_generator, batch_size=128)
print("test loss, test acc:", results)


Evaluate on test data
21/21 [==============================] - 197s 10s/step - loss: 0.5379 - acc: 0.7623
test loss, test acc: [0.5379151701927185, 0.762269914150238]


In [64]:
small_test_generator = test_datagen.flow_from_directory(
        small_test,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle=False)

Found 6 images belonging to 2 classes.


In [86]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = restored_keras_model.predict_classes(test_generator)
print("predictions shape:", predictions.shape)

Generate predictions for 3 samples


/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


predictions shape: (652, 1)


In [107]:
# Identify incorrect predictions
pred_clean = []
for i in range(len(predictions)):
  pred_clean.append(predictions[i][0])

In [71]:
test_generator.class_indices

{'happy': 0, 'sad': 1}

In [109]:
preds = list(zip(test_generator.filenames,pred_clean,test_generator.labels))

In [82]:
import pandas as pd

In [110]:
pred_df = pd.DataFrame(preds,columns=['image','prediction','actual'])

In [122]:
mask = pred_df['prediction'] != pred_df['actual']

In [128]:
image_list = []
for image in pred_df[mask]['image']:
  image_list.append(image)

In [140]:
happy = image_list[:60]
sad = image_list[60:]

In [156]:
base_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small'
# Directories for our training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
# os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
# os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)


test_happy_wrong = os.path.join(test_dir, 'happy_as_sad')
#os.mkdir(test_happy_wrong)

test_sad_wrong = os.path.join(test_dir, 'sad_as_happy')
#os.mkdir(test_sad_wrong)

In [153]:
original_dataset_dir = '/content/grive/My Drive/Metis/Deep-Learning/deep-learning-project/photos/small/'
test_happy = os.path.join(original_dataset_dir, 'test')
test_sad = os.path.join(original_dataset_dir,'test')

In [152]:
for fname in happy:
    src = os.path.join(test_happy, fname)
    dst = os.path.join(test_happy_wrong, fname)
    shutil.copyfile(src, dst)

In [158]:
for fname in sad:
    src = os.path.join(test_sad, fname)
    dst = os.path.join(test_sad_wrong, fname)
    shutil.copyfile(src, dst)

In [121]:
pred_df['image']

0.7622699386503068